In [4]:
import os
import numpy as np

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import sys
sys.path.append('../')
from utils.tools import get_config, default_loader, is_image_file, normalize
from data.dataset import Dataset
from tensorboardX import SummaryWriter 
import matplotlib.pyplot as plt
import torchvision.utils as vutils

from networks import autoencoder

In [5]:
class MVTecDataset(Dataset):
    def __init__(self, TYPE='bottle', isTrain=True):
        self.gt_path = '../MVTec/'+TYPE+'/ground_truth_resize/all'
        self.train_path = '../MVTec/'+TYPE+'/train_resize/train'
        self.test_path = '../MVTec/'+TYPE+'/test_resize/all'
        
        self.data_path = self.train_path if isTrain else self.test_path
        self.samples = [x for x in os.listdir(self.data_path) if is_image_file(x)]
        self.isTrain = isTrain
    
    def __getitem__(self, index):
        path = os.path.join(self.data_path, self.samples[index])
        
        img = default_loader(path)
        img = transforms.ToTensor()(img)  # turn the image to a tensor
        
        if not self.isTrain:
            gt_path = os.path.join(self.gt_path, self.samples[index])
        
            mask = default_loader(gt_path)
            mask = transforms.ToTensor()(mask)  # turn the image to a tensor
            return img, mask
        return img
    
    
    def __len__(self):
        return len(self.samples)

In [6]:
TYPES = ["bottle", "cable", "capsule", "carpet", "grid", "hazelnut", "leather", "metal_nut", "pill", "screw", "tile", "toothbrush", "transistor", "wood", "zipper"]

for index, TYPE in enumerate(TYPES):
    num_epochs = 100
    batch_size = 8
    lr = 1e-3
    weight_decay = 1e-5
    writer = SummaryWriter('checkpoint_AE2/exp-{}'.format(index))

    trainDatset = MVTecDataset(TYPE=TYPE, isTrain=True)
    testDatset = MVTecDataset(TYPE=TYPE, isTrain=False)
    train_loader = DataLoader(
        dataset=trainDatset,
        batch_size=batch_size, 
        shuffle=True,
        num_workers=4
    )
    test_loader = DataLoader(
        dataset=testDatset,
        batch_size=1, 
        shuffle=True,
        num_workers=4
    )
    model = nn.DataParallel(autoencoder.Autoencoder()).cuda()
    
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(
        model.parameters(), 
        lr=lr,
        weight_decay=weight_decay
    )

    for epoch in range(num_epochs):
        for img in train_loader:
            img = Variable(img).cuda()

            # ===================forward=====================
            output = model(img)
            loss = criterion(output, img)
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
        writer.add_scalar('loss', loss, epoch)

        if epoch % 5 == 0:
            writer.add_images('Reconstruct', output, epoch)
    
    for (img, mask) in test_loader:
        img = Variable(img).cuda()
        output = model(img)
        
        writer.add_images('Test Origin', img, epoch)
        writer.add_images('Test reconstruce', output, epoch)

epoch [1/100], loss:0.2398
epoch [2/100], loss:0.2342
epoch [3/100], loss:0.2250
epoch [4/100], loss:0.2214
epoch [5/100], loss:0.2140
epoch [6/100], loss:0.2081
epoch [7/100], loss:0.2048
epoch [8/100], loss:0.1933
epoch [9/100], loss:0.1896
epoch [10/100], loss:0.1868
epoch [11/100], loss:0.1802
epoch [12/100], loss:0.1767
epoch [13/100], loss:0.1715
epoch [14/100], loss:0.1671
epoch [15/100], loss:0.1592
epoch [16/100], loss:0.1612
epoch [17/100], loss:0.1551
epoch [18/100], loss:0.1526
epoch [19/100], loss:0.1469
epoch [20/100], loss:0.1428
epoch [21/100], loss:0.1340
epoch [22/100], loss:0.1365
epoch [23/100], loss:0.1313
epoch [24/100], loss:0.1281
epoch [25/100], loss:0.1222
epoch [26/100], loss:0.1189
epoch [27/100], loss:0.1137
epoch [28/100], loss:0.1154
epoch [29/100], loss:0.1118
epoch [30/100], loss:0.1081
epoch [31/100], loss:0.1074
epoch [32/100], loss:0.1002
epoch [33/100], loss:0.1016
epoch [34/100], loss:0.0980
epoch [35/100], loss:0.0936
epoch [36/100], loss:0.0909
e

FileNotFoundError: [Errno 2] No such file or directory: '../MVTec/transistor/test_resize/all'

In [ ]:
import sys
sys.path.append('./PerceptualSimilarity')

import models as PerceptualSimilarity
from sklearn.metrics import roc_auc_score, accuracy_score

In [21]:
def saveSaparateDif(d, idx):
    # 視覺化不同層的 feature 的重要性
    ironman_grid = plt.GridSpec(2, 3, wspace=0.4, hspace=0.3)

    fig = plt.figure(figsize=(20,10))
    
    for index in range(5):
        ax = fig.add_subplot(ironman_grid[int(index / 3), index % 3])
        im = ax.imshow(d[index][0].cpu().detach().numpy().transpose((1, 2, 0)).squeeze(), cmap="Blues")
        plt.colorbar(im, extend='both')
        im.set_clim(0, 0.1)

    plt.savefig('./AE_result/' + str(idx) + '_saparate.png')
    plt.close(fig)

def difNormalize(input_matrix, threshold=None):
    _min = input_matrix.min()
    _max = input_matrix.max()
    
    input_matrix = (input_matrix - _min) / (_max - _min)
    
    if threshold != None:
        input_matrix[input_matrix < threshold] = 0
        input_matrix[input_matrix >= threshold] = 1
        
    return input_matrix

def denormalize(input_matrix):
    return np.around(input_matrix * 127.5 + 127.5).astype(int)

In [22]:
perceptual_loss = PerceptualSimilarity.PerceptualLoss(model='net-lin', net='alex', use_gpu=True, gpu_ids=[0])
L2_loss = nn.MSELoss(reduction='none')

total_auc = 0
total_acc = 0
total_image = 0
for index, (img, mask) in enumerate(test_loader):
    img = Variable(img).cuda()
    mask = Variable(mask).cuda()

    # 取得 model 輸出
    output = model(img)

    # 計算 dif (相似度以及 L2)
    dif, separateDif = perceptual_loss.forward(output, img)
    l1Dif = L2_loss(output, img)
    l1Dif = torch.mean(l1Dif, 1, True)
    
    maskEdge = difNormalize(dif[0].cpu().detach().numpy() * l1Dif[0].cpu().detach().numpy())
    maskEdge2 = difNormalize(dif[0].cpu().detach().numpy() * l1Dif[0].cpu().detach().numpy(), threshold=0.5)
    
    mask = torch.mean(mask, 1, True)
    true_mask = mask[0].cpu().detach().numpy().flatten()

    AUC = roc_auc_score(true_mask, maskEdge.flatten())
    ACC = accuracy_score(true_mask, maskEdge2.flatten())
    
    total_auc += AUC
    total_acc += ACC
    total_image += 1
    
    writer.add_image('Test reconstruct', output[0], epoch)

print("Acerage AUC = {}".format(total_auc / total_image))
print("Acerage ACC = {}".format(total_acc / total_image))
    
#     saveSaparateDif(separateDif, index)
#     # 畫圖
#     ironman_grid = plt.GridSpec(2, 3, wspace=0.4, hspace=0.3)

#     fig = plt.figure(figsize=(20,10))
#     ax = fig.add_subplot(ironman_grid[:, :2])
#     im = ax.imshow( difNormalize(dif[0].cpu().detach().numpy() * l1Dif[0].cpu().detach().numpy() ).transpose((1, 2, 0)).squeeze() , cmap="Blues")
#     plt.colorbar(im, extend='both')
#     im.set_clim(0, 0.1)

#     ax2 = fig.add_subplot(ironman_grid[0, 2])
#     ax2.imshow(img[0].cpu().detach().numpy().transpose((1, 2, 0)))


#     ax3 = fig.add_subplot(ironman_grid[1, 2])
#     ax3.imshow(output[0].cpu().detach().numpy().transpose((1, 2, 0)))

#     plt.savefig('./AE_result/'+str(index)+'_result.png')
#     plt.close(fig)

Setting up Perceptual loss...
Loading model from: /workspace/PerceptualSimilarity/models/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


FileNotFoundError: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-14-dac4e0e8c11f>", line 20, in __getitem__
    mask = default_loader(gt_path)
  File "/workspace/utils/tools.py", line 18, in default_loader
    return pil_loader(path)
  File "/workspace/utils/tools.py", line 12, in pil_loader
    with open(path, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: './MVTec/grid/ground_truth_resize/all/57.png'
